# WIP: Diabetes Dataset Analysis 

In [16]:
import pandas as pd
import pandas_profiling
from sklearn import datasets

In [2]:
dataset = datasets.load_diabetes()

What attributes does the object have:

In [12]:
dir(dataset)

['DESCR', 'data', 'feature_names', 'target']

Standard attributes for a scikit dataset. Let’s see what the notes on the dataset are:

In [3]:
print(dataset.DESCR)

Diabetes dataset

Notes
-----

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

Data Set Characteristics:

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attributes:
    :Age:
    :Sex:
    :Body mass index:
    :Average blood pressure:
    :S1:
    :S2:
    :S3:
    :S4:
    :S5:
    :S6:

Note: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).

Source URL:
http://www4.stat.ncsu.edu/~boos/var.select/diabetes.html

For more information see:
Bradley Efron, Trevor Hastie, Iain Johnstone and Robert Tibshirani

To summarise the data:
- Only quantitative variables
- Data has been centred and scaled
- y is also quantitative
- six sereum measurements

In [6]:
dataset.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

Convert the numpy matrix to a data frame object

In [13]:
df = pd.DataFrame(
    data=dataset.data, 
    index=None, 
    columns=dataset.feature_names
)

Add the target column:

In [14]:
df['y'] = dataset.target

In [19]:
df.head(n=5)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,y
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


What does the profile of the data look like:

In [18]:
pf = pandas_profiling.ProfileReport(df)

In [20]:
pf

Number of variables,11
Number of observations,442
Total Missing (%),0.0%
Total size in memory,38.0 KiB
Average record size in memory,88.1 B
Numeric,10
Categorical,0
Boolean,1
Date,0
Text (Unique),0
Rejected,0
